## Attempt to get Tripifier working

In [63]:
%load_ext autoreload
%autoreload 2

from typing import List, Dict
from google.transit import gtfs_realtime_pb2 as gtfs_rt
from protobuf_to_dict import protobuf_to_dict
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re
from datetime import datetime

from pandas.io.json import json_normalize
from itertools import groupby

import GTFS_Utils as gu

In [4]:
def parse_trip_updates(realtime_data: List[Dict]) -> pd.DataFrame:
    # Get rid of any vehicle update rows
    realtime_data = [d for d in realtime_data if 'vehicle' not in d]    
    df = json_normalize(realtime_data)
    
    # Get rid of null stop_time_updates
    #df = df[~df['trip_update.stop_time_update'].isnull()]

    idx = df.index.repeat(df['trip_update.stop_time_update'].str.len())
    df1 = pd.DataFrame({'trip_update.stop_time_update': np.concatenate(df['trip_update.stop_time_update'].values)})
    df1.index = idx
    df = df1.join(df.drop('trip_update.stop_time_update', axis=1), how='left')
    df = df.reset_index(drop=True)
    d2 = json_normalize(df['trip_update.stop_time_update'])
    df = pd.concat([d2, df.drop('trip_update.stop_time_update', axis=1)], axis=1)
    df = df.drop(list(df.filter(regex='vehicle')), axis=1) 
    return df

In [57]:
import gtfs_tripify as gt
from gtfs_tripify import tripify as tp

path = '201906/20190601/gtfs_ace_20190601_041956.gtfs'
path2 = '20190601/gtfs_ace_20190601_'

feed1 = gtfs_rt.FeedMessage()
feed2 = gtfs_rt.FeedMessage()

log = None
with open(path, 'rb') as file:
    raw_str = file.read()
    feed1.ParseFromString(raw_str)
    dt = tp.dictify(feed1)    
    log = gt.logify(dt)
    
#dt1 = protobuf_to_dict(feed1)
#dt2 = protobuf_to_dict(feed2)

#gt.logify([dt1, dt2])    

KeyError: 0

TypeError: dictify() missing 1 required positional argument: 'buffer'

In [41]:
import gtfs_tripify as gt
from zipfile import ZipFile
import os

# Update this value with the path to the GTFS-RT rollup on your local machine.
DOWNLOAD_URL = '/content/jupyter/201906.zip'

z = ZipFile(DOWNLOAD_URL)
z.extract('20190601.zip')
z.extract('20190612.zip')

messages = []
# filter out non-GTFS files
for filename in sorted(os.listdir('.')):
    if '.py' not in filename and 'gtfs_7_' in filename:
        with open(filename, 'rb') as f:
            messages.append(f.read())

# build the logbooks
first_logbook, first_logbook_timestamps, _ = gt.logify(messages[:len(messages) // 2])
second_logbook, second_logbook_timestamps, _ = gt.logify(messages[len(messages) // 2:])

# merge the logbooks
logbook = gt.ops.merge_logbooks(
    [(first_logbook, first_logbook_timestamps), (second_logbook, second_logbook_timestamps)]    
)

# save to disk
gt.ops.to_csv(logbook, 'logbook.csv')

({}, {})

'20190629'